In [1]:
import pandas as pd
import numpy as np
import os

# from pyecharts.globals import CurrentConfig, NotebookType
# CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

from pyecharts.faker import Faker
from pyecharts.charts import Bar
from pyecharts import options as opts

In [3]:
# CurrentConfig.ONLINE_HOST

In [2]:
pd.options.display.float_format = '{:.2f}'.format 

# 0. 데이터 열기 (0.3, 0.4만 실행)

## 0.3. 전국 행정구역 코드 열기

In [81]:
cd_df = pd.read_excel('./전국_누적_행정동코드_정의서_KT_20230802.xlsx', engine = 'openpyxl')
cd_df.head()

,sido_cd,sido_nm,sgg_cd,sgg_nm,admdong_cd,admdong_nm
0,11,서울특별시,11110,종로구,11110515,청운효자동
1,11,서울특별시,11110,종로구,11110530,사직동
2,11,서울특별시,11110,종로구,11110540,삼청동
3,11,서울특별시,11110,종로구,11110550,부암동
4,11,서울특별시,11110,종로구,11110560,평창동


## 0.4. 최종 데이터 열기

In [3]:
df = pd.read_csv('./이동목적별_이동인구_수원시_팔달구_인계동.csv')
df.head()

,ETL_YMD,SEX_TYPE_ITG_CD,AGE_GR,IN_FORN_DIV_NM,O_TIME_CD,D_TIME_CD,O_ADMI_CD,D_ADMI_CD,PURPOSE,AVG_DIST_METER,AVG_MOVE_TIME_MINUT,CNT
0,20230624,M,25,내국인,20,21,41115730.00,41271560.00,기타,14063.00,3650,*
1,20230624,F,35,내국인,15,16,41115730.00,41410580.00,기타,12166.00,3095,3.997136789997981
2,20230630,F,35,내국인,11,11,41115710.00,41115730.00,기타,1570.00,283,3.99497895730631
3,20230616,F,25,내국인,19,19,41115730.00,41117610.00,기타,3978.00,2496,*
4,20230627,M,20,내국인,23,23,41113670.00,41115730.00,귀가,461.00,50,3.7491279117180096


In [4]:
# 컬럼명 소문자로 바꾸기
df.columns = [col.lower() for col in df.columns.tolist()]
df.head()

,etl_ymd,sex_type_itg_cd,age_gr,in_forn_div_nm,o_time_cd,d_time_cd,o_admi_cd,d_admi_cd,purpose,avg_dist_meter,avg_move_time_minut,cnt
0,20230624,M,25,내국인,20,21,41115730.00,41271560.00,기타,14063.00,3650,*
1,20230624,F,35,내국인,15,16,41115730.00,41410580.00,기타,12166.00,3095,3.997136789997981
2,20230630,F,35,내국인,11,11,41115710.00,41115730.00,기타,1570.00,283,3.99497895730631
3,20230616,F,25,내국인,19,19,41115730.00,41117610.00,기타,3978.00,2496,*
4,20230627,M,20,내국인,23,23,41113670.00,41115730.00,귀가,461.00,50,3.7491279117180096


# 1. 데이터 EDA

- 각 컬럼별로 row 수 확인

## 1.1 날짜

In [5]:
print(sorted(df['etl_ymd'].unique()))
print('---')
print(len(df['etl_ymd'].unique()))


[20230601, 20230602, 20230603, 20230604, 20230605, 20230606, 20230607, 20230608, 20230609, 20230610, 20230611, 20230612, 20230613, 20230614, 20230615, 20230616, 20230617, 20230618, 20230619, 20230620, 20230621, 20230622, 20230623, 20230624, 20230625, 20230626, 20230627, 20230628, 20230629, 20230630]
---
30


In [6]:
df_date = pd.DataFrame(df['etl_ymd'].value_counts()).reset_index()
df_date = df_date.sort_values('etl_ymd').reset_index(drop = True)

In [7]:
df_date.columns = ['etl_ymd', 'count']

In [8]:
bar = (
    Bar()
    .add_xaxis(df_date['etl_ymd'].tolist())
    .add_yaxis('count', df_date['count'].tolist())
    # render will generate a local HTML file, by default render.html will be generated in the current directory
    # You can also pass in a path parameter, e.g. bar.render("mycharts.html")
    # bar.render()
    .set_global_opts(title_opts=opts.TitleOpts(title="날짜별 데이터 건수", subtitle="유입, 유출량이 아닌 단순 row 수"))
    )



## 추가 부분
bar.load_javascript()
## 
bar.render_notebook()

## 1.2. 성별

In [9]:
# 성별
df['sex_type_itg_cd'].unique()

array(['M', 'F'], dtype=object)

In [10]:
df_sex = pd.DataFrame(df['sex_type_itg_cd'].value_counts()).reset_index()
df_sex.columns = ['sex_type_itg_cd', 'count']

In [11]:
bar = (
    Bar()
    .add_xaxis(df_sex['sex_type_itg_cd'].tolist())
    .add_yaxis('count', df_sex['count'].tolist())
    # render will generate a local HTML file, by default render.html will be generated in the current directory
    # You can also pass in a path parameter, e.g. bar.render("mycharts.html")
    # bar.render()
    .set_global_opts(title_opts=opts.TitleOpts(title="성별 건수", subtitle="단순 row 수"))
    )



## 추가 부분
bar.load_javascript()
## 
bar.render_notebook()

## 1.3. 연령

In [12]:
df['age_gr'].unique()

array([25, 35, 20, 50, 55, 30, 60, 40, 15, 10, 45,  0], dtype=int64)

In [13]:
df_age = pd.DataFrame(df['age_gr'].value_counts()).reset_index()
df_age = df_age.sort_values('age_gr').reset_index(drop = True)

In [14]:
df_age.columns = ['age_gr', 'count']

In [15]:
bar = (
    Bar()
    .add_xaxis(df_age['age_gr'].tolist())
    .add_yaxis('count', df_age['count'].tolist())
    # render will generate a local HTML file, by default render.html will be generated in the current directory
    # You can also pass in a path parameter, e.g. bar.render("mycharts.html")
    # bar.render()
    .set_global_opts(title_opts=opts.TitleOpts(title="연령대별 건수", subtitle="단순 row 수"))
    )



## 추가 부분
bar.load_javascript()
## 
bar.render_notebook()

In [16]:
def make_EDA_bar(df, col, title):
    bar = (
        Bar()
        .add_xaxis(df[col].tolist())
        .add_yaxis('count', df['count'].tolist())
        # render will generate a local HTML file, by default render.html will be generated in the current directory
        # You can also pass in a path parameter, e.g. bar.render("mycharts.html")
        # bar.render()
        .set_global_opts(title_opts=opts.TitleOpts(title=f"{title}", subtitle="단순 row 수"))
        )
    
    
    
    ## 추가 부분
    bar.load_javascript()
    ## 
    return bar.render_notebook()

## 1.4 내국인, 외국인

In [17]:
df['in_forn_div_nm'].unique()

array(['내국인', '외국인'], dtype=object)

In [18]:
df_forn = pd.DataFrame(df['in_forn_div_nm'].value_counts()).reset_index()
df_forn.columns = ['in_forn_div_nm', 'count']

In [19]:
make_EDA_bar(df_forn, 'in_forn_div_nm', '내국인, 외국인 건수')

# 1.5 출발시간, 도착시간

In [20]:
# 출발시간
df_otime = pd.DataFrame(df['o_time_cd'].value_counts()).reset_index()
df_otime = df_otime.sort_values('o_time_cd').reset_index(drop = True)
df_otime.columns = ['o_time_cd', 'count']

# 도착시간
df_dtime = pd.DataFrame(df['d_time_cd'].value_counts()).reset_index()
df_dtime = df_dtime.sort_values('d_time_cd').reset_index(drop = True)
df_dtime.columns = ['d_time_cd', 'count']

In [21]:
make_EDA_bar(df_otime, 'o_time_cd', '출발시간별 건수')

In [22]:
make_EDA_bar(df_dtime, 'd_time_cd', '도착시간별 건수')

## 1.6 목적별 건수

In [23]:
print(df['purpose'].unique())
df_purpose = pd.DataFrame(df['purpose'].value_counts()).reset_index()

['기타' '귀가' '출근' '귀사' '관광' '등교' '쇼핑' '병원']


In [25]:
df_purpose.columns = ['purpose', 'count']

In [26]:
make_EDA_bar(df_purpose, 'purpose', '목적별 건수')

## 1.7. 평균이동거리 (ASP에는 없는 컬럼)

In [27]:
# 요약
# 건수, 평균, 표준편차, 최소값, 1사분위값, 중앙값, 3사분위값, 최대값 
df['avg_dist_meter'].describe()

count   1706674.00
mean      11789.63
std       24640.26
min          50.00
25%        1498.00
50%        3897.00
75%       12820.00
max      453195.00
Name: avg_dist_meter, dtype: float64

In [28]:
# 데이터 10개 구간으로 나누기
df['range'] = pd.cut(df['avg_dist_meter'], bins = 10)

# 도수분포
frequency_table = df['range'].value_counts().sort_index()

# 도수분포표를 표 형태로 변환
frequency_table = frequency_table.reset_index()
frequency_table.columns = ['Range', 'Frequency']

# 안쓰는 컬럼 지우기
df = df.drop('range', axis = 1)

In [29]:
# 결과 출력
frequency_table

,Range,Frequency
0,"(-403.145, 45364.5]",1642590
1,"(45364.5, 90679.0]",41333
2,"(90679.0, 135993.5]",8566
3,"(135993.5, 181308.0]",5720
4,"(181308.0, 226622.5]",2902
5,"(226622.5, 271937.0]",2430
6,"(271937.0, 317251.5]",2659
7,"(317251.5, 362566.0]",47
8,"(362566.0, 407880.5]",1
9,"(407880.5, 453195.0]",426


In [30]:
# 히스토그램
bar = (
    Bar()
    .add_xaxis(frequency_table.Range.astype(str).tolist())  # 구간을 문자열로 변환하여 x축 라벨로 사용
    .add_yaxis("Frequency", frequency_table.Frequency.tolist())
    .set_global_opts(title_opts=opts.TitleOpts(title="평균이동거리 히스토그램"))
)

bar.render_notebook()

## 1.8. 평균 이동 시간 (ASP에는 없는 컬럼)

In [31]:
# 요약
# 건수, 평균, 표준편차, 최소값, 1사분위값, 중앙값, 3사분위값, 최대값 
df['avg_move_time_minut'].describe()

count   1706674.00
mean       2972.21
std        3990.34
min           0.00
25%         780.00
50%        1886.00
75%        3679.00
max       82247.00
Name: avg_move_time_minut, dtype: float64

In [32]:
# 데이터 10개 구간으로 나누기
df['range'] = pd.cut(df['avg_move_time_minut'], bins = 10)

# 도수분포
frequency_table = df['range'].value_counts().sort_index()

# 도수분포표를 표 형태로 변환
frequency_table = frequency_table.reset_index()
frequency_table.columns = ['Range', 'Frequency']

# 안쓰는 컬럼 지우기
df = df.drop('range', axis = 1)

In [33]:
# 결과 출력
frequency_table

,Range,Frequency
0,"(-82.247, 8224.7]",1601691
1,"(8224.7, 16449.4]",79676
2,"(16449.4, 24674.1]",15916
3,"(24674.1, 32898.8]",5088
4,"(32898.8, 41123.5]",2305
5,"(41123.5, 49348.2]",1171
6,"(49348.2, 57572.9]",504
7,"(57572.9, 65797.6]",214
8,"(65797.6, 74022.3]",92
9,"(74022.3, 82247.0]",17


In [34]:
# 히스토그램
bar = (
    Bar()
    .add_xaxis(frequency_table.Range.astype(str).tolist())  # 구간을 문자열로 변환하여 x축 라벨로 사용
    .add_yaxis("Frequency", frequency_table.Frequency.tolist())
    .set_global_opts(title_opts=opts.TitleOpts(title="평균이동시간(분) 히스토그램"))
)

bar.render_notebook()

## 1.9 건수

In [35]:
# 숫자가 아닌 값
print('숫자가 아닌 값 개수: ', len(df[df['cnt'] == '*']))
print('총 데이터 건수: ', len(df))
print('숫자가 아닌 값 비율: ', round(len(df[df['cnt'] == '*'])*100/len(df), 2), '%')

숫자가 아닌 값 개수:  649691
총 데이터 건수:  1706674
숫자가 아닌 값 비율:  38.07 %


In [36]:
df_cnt = df[df['cnt'] != '*'].reset_index(drop = True)
df_cnt['cnt'] = df_cnt['cnt'].astype(float)

In [37]:
# 요약
# 건수, 평균, 표준편차, 최소값, 1사분위값, 중앙값, 3사분위값, 최대값 
df_cnt['cnt'].describe()

count   1056983.00
mean          5.38
std           5.58
min           3.04
25%           3.45
50%           3.75
75%           5.10
max         176.43
Name: cnt, dtype: float64

In [38]:
# 데이터 10개 구간으로 나누기
df_cnt['range'] = pd.cut(df_cnt['cnt'], bins = 10)

# 도수분포
frequency_table = df_cnt['range'].value_counts().sort_index()

# 도수분포표를 표 형태로 변환
frequency_table = frequency_table.reset_index()
frequency_table.columns = ['Range', 'Frequency']

# 안쓰는 컬럼 지우기
df_cnt = df_cnt.drop('range', axis = 1)

In [39]:
# 결과 출력
frequency_table

,Range,Frequency
0,"(2.863, 20.376]",1036324
1,"(20.376, 37.715]",12932
2,"(37.715, 55.055]",5084
3,"(55.055, 72.394]",1875
4,"(72.394, 89.734]",534
5,"(89.734, 107.073]",157
6,"(107.073, 124.413]",52
7,"(124.413, 141.752]",24
8,"(141.752, 159.092]",0
9,"(159.092, 176.431]",1


In [40]:
# 히스토그램
bar = (
    Bar()
    .add_xaxis(frequency_table.Range.astype(str).tolist())  # 구간을 문자열로 변환하여 x축 라벨로 사용
    .add_yaxis("Frequency", frequency_table.Frequency.tolist())
    .set_global_opts(title_opts=opts.TitleOpts(title="이동건수(cnt) 히스토그램"))
)

bar.render_notebook()

# 2. 이동 분석 - 유입유출 건수(cnt) 중심으로

### cnt가 숫자가 아닌 row는 삭제하고 진행

In [45]:
datadf = df[df['cnt'] != '*'].reset_index(drop = True)
datadf['cnt'] = datadf['cnt'].astype(float)
print(len(df))
print(len(datadf))

1706674
1056983


### 바차트 함수

In [46]:
def make_bar(df, xcol, ycol, title, subtitle):
    bar = (
        Bar()
        .add_xaxis(df[xcol].tolist())
        .add_yaxis(f'{ycol}', df[ycol].tolist())
        # render will generate a local HTML file, by default render.html will be generated in the current directory
        # You can also pass in a path parameter, e.g. bar.render("mycharts.html")
        # bar.render()
        .set_global_opts(title_opts=opts.TitleOpts(title=f'{title}', subtitle=f'{subtitle}'))
        )
    
    
    
    ## 추가 부분
    bar.load_javascript()
    ## 
    return bar.render_notebook()

In [47]:
def make_groupbar(df, xcol, ycols, title, subtitle):
    bar = Bar()
    # x축 데이터 추가
    bar.add_xaxis(df[xcol].tolist())
    
    # 여러 y축 값(시리즈) 추가
    for ycol in ycols:
        bar.add_yaxis(f'{ycol}', df[ycol].tolist())
    
    # 글로벌 옵션 설정
    bar.set_global_opts(title_opts=opts.TitleOpts(title=f'{title}', subtitle=f'{subtitle}'),
                        datazoom_opts = opts.DataZoomOpts()
                       )
    
    ## 추가 부분
    bar.load_javascript()
    ## 
    return bar.render_notebook()

## 2.0 유입유출 구분

In [48]:
datadf.loc[datadf['o_admi_cd'] == 41115730, '이동구분'] = '유출'
datadf.loc[datadf['d_admi_cd'] == 41115730, '이동구분'] = '유입'
datadf.loc[(datadf['d_admi_cd'] == 41115730) & (datadf['o_admi_cd'] == 41115730), '이동구분'] = '동일지역간이동'

In [49]:
df_cate = datadf.groupby('이동구분')[['cnt']].sum()
df_cate = round(df_cate, 2).reset_index()

In [50]:
make_bar(df_cate, '이동구분', 'cnt', '일별 이동건수 합계', '유입, 유출 합계')

## 2.1. 날짜별 분석

In [51]:
df_date = datadf.pivot_table(index = 'etl_ymd', columns = '이동구분', values = 'cnt', aggfunc = 'sum')
df_date = round(df_date, 2)
df_date = df_date.reset_index()

In [52]:
legend = ['유입', '유출', '동일지역간이동']

In [53]:
make_groupbar(df_date, 'etl_ymd', legend, '일별 이동건수 합계', '유입, 유출 합계')

## 2.2. 요일별 분석

In [54]:
from datetime import datetime
datadf['dow'] = datadf.etl_ymd.apply(lambda x : datetime.strptime(str(x), '%Y%m%d').weekday())
datadf['dow'] = datadf['dow'].replace({0:'월', 1:'화', 2:'수', 3:'목', 4:'금', 5:'토', 6:'일'})

In [55]:
df_dow = datadf.pivot_table(index = 'dow', columns = '이동구분', values = 'cnt', aggfunc = 'sum')
df_dow = round(df_dow, 2)
df_dow = df_dow.reset_index()

In [56]:
# 요일 순서 정의 (월요일부터 일요일까지)
dow_order = ['월', '화', '수', '목', '금', '토', '일']

# 요일 순서에 따라 데이터프레임 정렬
df_dow['dow'] = pd.Categorical(df_dow['dow'], categories=dow_order, ordered=True)
df_sorted = df_dow.sort_values('dow')
df_sorted = df_sorted.reset_index(drop = True)

In [57]:
make_groupbar(df_sorted, 'dow', legend,  '요일별 이동건수 합계', '유입, 유출 합계')

## 2.3. 시간대별 분석 (출발시간대, 도착시간대)

In [58]:
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.faker import Faker

In [59]:
df_otime = datadf.pivot_table(index = 'o_time_cd', columns = '이동구분', values = 'cnt', aggfunc = 'sum')
df_dtime = datadf.pivot_table(index = 'd_time_cd', columns = '이동구분', values = 'cnt', aggfunc = 'sum')

In [60]:
df_otime = round(df_otime, 2).reset_index()
df_dtime = round(df_dtime, 2).reset_index()

In [61]:
# 출발 시간대별 이동건수 합계
c = (
    Line()
    .add_xaxis(df_otime['o_time_cd'].tolist())
    .add_yaxis(
        "유입",
        df_otime['유입'].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )
    .add_yaxis(
        "유출",
        df_otime['유출'].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )
    .add_yaxis(
        "동일지역간이동",
        df_otime['동일지역간이동'].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="출발 시간대별 이동건수 합계", subtitle='유입, 유출 합계'),
                     datazoom_opts = opts.DataZoomOpts())
    # .render("line_markpoint.html")
)
c.render_notebook()

In [62]:
# 도착 시간대별 이동건수 합계
c = (
    Line()
    .add_xaxis(df_otime['o_time_cd'].tolist())
    .add_yaxis(
        "유입",
        df_otime['유입'].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )
    .add_yaxis(
        "유출",
        df_otime['유출'].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )
    .add_yaxis(
        "동일지역간이동",
        df_otime['동일지역간이동'].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="도착 시간대별 이동건수 합계", subtitle='유입, 유출 합계'),
                     datazoom_opts = opts.DataZoomOpts())
    # .render("line_markpoint.html")
)
c.render_notebook()

## 2.4. 목적별 유입유출

In [63]:
df_purpose = datadf.pivot_table(index = 'purpose', columns = '이동구분', values = 'cnt', aggfunc = 'sum')
df_purpose = round(df_purpose, 2)

In [64]:
df_purpose['total'] = df_purpose.sum(axis = 1)
df_purpose = df_purpose.sort_values('total', ascending = False).reset_index()
df_purpose

이동구분,purpose,동일지역간이동,유입,유출,total
0,기타,616467.68,1469232.40,1065759.40,3151459.48
1,귀가,363680.29,494853.63,822497.15,1681031.07
2,출근,143579.15,268796.75,214065.57,626441.47
3,귀사,42901.65,29275.71,31625.73,103803.09
4,등교,15995.56,34428.11,52982.58,103406.25
5,쇼핑,nan,nan,11507.48,11507.48
6,관광,nan,nan,7243.99,7243.99
7,병원,nan,nan,2809.49,2809.49


In [65]:
# df, xcol, ycol1, ycol2, title, subtitle
bar = (
    Bar()
    .add_xaxis(df_purpose['purpose'].tolist())
    .add_yaxis('유입', df_purpose['유입'].tolist())
    .add_yaxis('유출', df_purpose['유출'].tolist())
    .add_yaxis('동일지역간이동', df_purpose['유출'].tolist())
    # render will generate a local HTML file, by default render.html will be generated in the current directory
    # You can also pass in a path parameter, e.g. bar.render("mycharts.html")
    # bar.render()
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f'목적별 이동건수 합계', subtitle=f'유입, 유출 합계')
        # datazoom_opts = opts.DataZoomOpts()
    )
)

bar.load_javascript()
bar.render_notebook()

## 2.5. 성별 이동

In [66]:
df_sex = datadf.pivot_table(index = '이동구분', columns = 'sex_type_itg_cd', values = 'cnt', aggfunc = 'sum')
df_sex = round(df_sex, 2).reset_index()
df_sex

sex_type_itg_cd,이동구분,F,M
0,동일지역간이동,622636.36,559987.99
1,유입,1193476.02,1103110.57
2,유출,1144164.11,1064327.29


In [67]:
make_groupbar(df_sex, '이동구분', ['M', 'F'], '성별 이동인구 합계', '유입, 유출 합계')

## 2.6. 연령별 이동

In [68]:
datadf['age'] = datadf['age_gr']//10*10

In [69]:
df_age = datadf.pivot_table(index = 'age', columns = '이동구분', values = 'cnt', aggfunc = 'sum')
df_age = round(df_age, 2).reset_index()
df_age

이동구분,age,동일지역간이동,유입,유출
0,0,15975.55,22680.91,22937.81
1,10,95053.05,247408.82,241872.73
2,20,255608.27,397104.55,371893.07
3,30,240490.90,422079.04,408115.70
4,40,211834.77,581323.36,568001.57
5,50,185378.23,366520.54,351409.08
6,60,178283.57,259469.38,244261.44


In [70]:
make_groupbar(df_age, 'age', legend, '연령별 이동인구 합계', '유입, 유출 합계')

## 2.7. 성연령대별 이동 (유입유출 따로)

In [71]:
datadf['sexage'] = datadf['sex_type_itg_cd']+(datadf['age_gr']//10*10).astype(str)

In [72]:
# 유입 유출 나눠서 표출
df_sexage_in = datadf[datadf['이동구분'] == '유입'].pivot_table(index = 'age', columns = 'sex_type_itg_cd', values = 'cnt', aggfunc = 'sum')
df_sexage_out = datadf[datadf['이동구분'] == '유출'].pivot_table(index = 'age', columns = 'sex_type_itg_cd', values = 'cnt', aggfunc = 'sum')
df_sexage_same = datadf[datadf['이동구분'] == '동일지역간이동'].pivot_table(index = 'age', columns = 'sex_type_itg_cd', values = 'cnt', aggfunc = 'sum')

df_sexage_in = round(df_sexage_in, 2).reset_index()
df_sexage_out = round(df_sexage_out, 2).reset_index()
df_sexage_same = round(df_sexage_same, 2).reset_index()

In [74]:
# df, xcol, ycol1, ycol2, title, subtitle
bar = (
    Bar()
    .add_xaxis(df_sexage_in['age'].tolist())
    .add_yaxis('남성', df_sexage_in['M'].tolist())
    .add_yaxis('여성', df_sexage_in['F'].tolist())
    
    # render will generate a local HTML file, by default render.html will be generated in the current directory
    # You can also pass in a path parameter, e.g. bar.render("mycharts.html")
    # bar.render()
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f'성연령별 유입건수 합계', subtitle=f'유입만')
        # datazoom_opts = opts.DataZoomOpts()
    )
)

bar.load_javascript()
bar.render_notebook()

In [75]:
# df, xcol, ycol1, ycol2, title, subtitle
bar = (
    Bar()
    .add_xaxis(df_sexage_out['age'].tolist())
    .add_yaxis('남성', df_sexage_out['M'].tolist())
    .add_yaxis('여성', df_sexage_out['F'].tolist())
    
    # render will generate a local HTML file, by default render.html will be generated in the current directory
    # You can also pass in a path parameter, e.g. bar.render("mycharts.html")
    # bar.render()
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f'성연령별 유출건수 합계', subtitle=f'유출만')
        # datazoom_opts = opts.DataZoomOpts()
    )
)

bar.load_javascript()
bar.render_notebook()

In [76]:
# df, xcol, ycol1, ycol2, title, subtitle
bar = (
    Bar()
    .add_xaxis(df_sexage_same['age'].tolist())
    .add_yaxis('남성', df_sexage_same['M'].tolist())
    .add_yaxis('여성', df_sexage_same['F'].tolist())
    
    # render will generate a local HTML file, by default render.html will be generated in the current directory
    # You can also pass in a path parameter, e.g. bar.render("mycharts.html")
    # bar.render()
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f'성연령별 동일지역 이동건수 합계', subtitle=f'동일지역 이동만')
        # datazoom_opts = opts.DataZoomOpts()
    )
)

bar.load_javascript()
bar.render_notebook()

In [77]:
# 같이
df_sexage = datadf.pivot_table(index = '이동구분', columns = 'sexage', values = 'cnt', aggfunc = 'sum')
df_sexage = round(df_sexage, 2).reset_index()

In [78]:
df_sexage

sexage,이동구분,F0,F10,F20,F30,F40,F50,F60,M0,M10,M20,M30,M40,M50,M60
0,동일지역간이동,9573.51,44362.75,136247.17,118374.84,110428.69,99943.49,103705.91,6402.05,50690.30,119361.10,122116.06,101406.08,85434.74,74577.66
1,유입,12131.96,117385.41,214012.93,199219.36,290898.47,200689.81,159138.09,10548.95,130023.41,183091.62,222859.68,290424.89,165830.73,100331.29
2,유출,12385.54,114715.48,199262.66,192102.16,283578.34,191046.86,151073.07,10552.27,127157.25,172630.41,216013.54,284423.23,160362.23,93188.37


## 2.8 이동지역별 인구 (유입)

### 2.8.1 시도

In [82]:
cd_df = cd_df.drop_duplicates('admdong_cd').reset_index(drop = True)

In [83]:
cd_df.columns

Index(['sido_cd', 'sido_nm', 'sgg_cd', 'sgg_nm', 'admdong_cd', 'admdong_nm'], dtype='object')

In [84]:
datadf = pd.merge(datadf, cd_df[['sido_nm', 'sgg_nm', 'admdong_nm', 'admdong_cd']], how = 'left', left_on = 'o_admi_cd', right_on = 'admdong_cd').rename(columns = {
    'sido_nm':'o_sido_nm',
    'sgg_nm':'o_sgg_nm',
    'admdong_nm':'o_admdong_nm'
}).drop('admdong_cd', axis = 1)
datadf = pd.merge(datadf, cd_df[['sido_nm', 'sgg_nm', 'admdong_nm', 'admdong_cd']], how = 'left', left_on = 'd_admi_cd', right_on = 'admdong_cd').rename(columns = {
    'sido_nm':'d_sido_nm',
    'sgg_nm':'d_sgg_nm',
    'admdong_nm':'d_admdong_nm'
}).drop('admdong_cd', axis = 1)

In [85]:
# 유입 시도별 (당연히.. 경기도...겠지...?)
df_osido = datadf[datadf['이동구분'] == '유입'].groupby('o_sido_nm')[['cnt']].sum()
df_osido = round(df_osido, 2).sort_values('cnt', ascending = False).reset_index()

# 유출 시도별
df_dsido = datadf[datadf['이동구분'] == '유출'].groupby('d_sido_nm')[['cnt']].sum()
df_dsido = round(df_dsido, 2).sort_values('cnt', ascending = False).reset_index()


In [87]:
df_osido.columns = ['지역', '유입']
df_dsido.columns = ['지역', '유출']
df_sido = pd.merge(df_osido, df_dsido, how = 'inner', on = '지역')

In [88]:
bar = Bar()
# x축 데이터 추가
bar.add_xaxis(df_sido['지역'].tolist())

# 여러 y축 값(시리즈) 추가
for ycol in ['유입', '유출']:
    bar.add_yaxis(f'{ycol}', df_sido[ycol].tolist())

# 글로벌 옵션 설정
bar.set_global_opts(title_opts=opts.TitleOpts(title=f'지역별 이동인구(시도)', subtitle=f'유입-해당지역으로부터 인계동으로 이동, 유출-인계동에서 해당지역으로 유출'),
                    # datazoom_opts = opts.DataZoomOpts()
                   )

## 추가 부분
bar.load_javascript()
## 
bar.render_notebook()

### 2.8.2. 시군구

In [89]:
# 유입 시군구별 
df_osgg = datadf[datadf['이동구분'] == '유입'].groupby('o_sgg_nm')[['cnt']].sum()
df_osgg = round(df_osgg, 2).sort_values('cnt', ascending = False).reset_index()

# 유출 시군구별
df_dsgg = datadf[datadf['이동구분'] == '유출'].groupby('d_sgg_nm')[['cnt']].sum()
df_dsgg = round(df_dsgg, 2).sort_values('cnt', ascending = False).reset_index()

df_osgg.columns = ['지역', '유입']
df_dsgg.columns = ['지역', '유출']
df_sgg = pd.merge(df_osgg, df_dsgg, how = 'inner', on = '지역')

In [90]:
bar = Bar()
# x축 데이터 추가
bar.add_xaxis(df_sgg['지역'].tolist())

# 여러 y축 값(시리즈) 추가
for ycol in ['유입', '유출']:
    bar.add_yaxis(f'{ycol}', df_sgg[ycol].tolist())

# 글로벌 옵션 설정
bar.set_global_opts(
    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30)),
    title_opts=opts.TitleOpts(title=f'지역별 이동인구(시군구)', subtitle=f'유입-해당지역으로부터 인계동으로 이동, 유출-인계동에서 해당지역으로 유출'),
                    datazoom_opts = opts.DataZoomOpts()
                   )

## 추가 부분
bar.load_javascript()
## 
bar.render_notebook()

In [91]:
df_sgg['지역'][:10]

0    수원시 영통구
1    수원시 권선구
2    수원시 팔달구
3        화성시
4    수원시 장안구
5    용인시 기흥구
6    용인시 수지구
7        오산시
8    성남시 분당구
9    용인시 처인구
Name: 지역, dtype: object

In [92]:
bar = Bar()
# x축 데이터 추가
bar.add_xaxis(df_sgg['지역'][:10].tolist())

# 여러 y축 값(시리즈) 추가
for ycol in ['유입', '유출']:
    bar.add_yaxis(f'{ycol}', df_sgg[ycol][:10].tolist())

# 글로벌 옵션 설정
bar.set_global_opts(
    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15)),
    title_opts=opts.TitleOpts(title=f'지역별 이동인구(시군구)', subtitle=f'유입-해당지역으로부터 인계동으로 이동, 유출-인계동에서 해당지역으로 유출'),
                    # datazoom_opts = opts.DataZoomOpts()
                   )

## 추가 부분
bar.load_javascript()
## 
bar.render_notebook()

In [93]:
# 유입인구 내림차순 
print('유입인구 순 내림차순')
print(df_sgg.sort_values('유입', ascending= False)['지역'][:10])
print('-----'*10)
print('유출인구 순 내림차순')
print(df_sgg.sort_values('유출', ascending= False)['지역'][:10])
print('순위 똑같음')

유입인구 순 내림차순
0    수원시 영통구
1    수원시 권선구
2    수원시 팔달구
3        화성시
4    수원시 장안구
5    용인시 기흥구
6    용인시 수지구
7        오산시
8    성남시 분당구
9    용인시 처인구
Name: 지역, dtype: object
--------------------------------------------------
유출인구 순 내림차순
0    수원시 영통구
1    수원시 권선구
2    수원시 팔달구
3        화성시
4    수원시 장안구
5    용인시 기흥구
6    용인시 수지구
7        오산시
8    성남시 분당구
9    용인시 처인구
Name: 지역, dtype: object
순위 똑같음


### 2.8.3. 읍면동

In [95]:
# 유입 시군구별 
df_odong = datadf[datadf['이동구분'] == '유입'].groupby(['o_sgg_nm', 'o_admdong_nm'])[['cnt']].sum()
df_odong = round(df_odong, 2).sort_values('cnt', ascending = False).reset_index()
df_odong['nm'] = df_odong['o_sgg_nm'] + ' ' + df_odong['o_admdong_nm']
df_odong = df_odong[['nm', 'cnt']]

# 유출 시군구별
df_ddong = datadf[datadf['이동구분'] == '유출'].groupby(['d_sgg_nm', 'd_admdong_nm'])[['cnt']].sum()
df_ddong = round(df_ddong, 2).sort_values('cnt', ascending = False).reset_index()
df_ddong['nm'] = df_ddong['d_sgg_nm'] + ' ' + df_ddong['d_admdong_nm']
df_ddong = df_ddong[['nm', 'cnt']]

df_odong.columns = ['지역', '유입']
df_ddong.columns = ['지역', '유출']
df_dong = pd.merge(df_odong, df_ddong, how = 'inner', on = '지역')

In [96]:
bar = Bar()
# x축 데이터 추가
bar.add_xaxis(df_dong['지역'][:20].tolist())

# 여러 y축 값(시리즈) 추가
for ycol in ['유입', '유출']:
    bar.add_yaxis(f'{ycol}', df_dong[ycol][:20].tolist())

# 글로벌 옵션 설정
bar.set_global_opts(
    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30)),
    title_opts=opts.TitleOpts(title=f'지역별 이동인구(읍면동)', subtitle=f'유입-해당지역으로부터 인계동으로 이동, 유출-인계동에서 해당지역으로 유출'),
                    datazoom_opts = opts.DataZoomOpts()
                   )

## 추가 부분
bar.load_javascript()
## 
bar.render_notebook()

In [97]:
# 유입인구 내림차순 
print('유입인구 순 내림차순')
print(df_odong.sort_values('유입', ascending= False)['지역'][:10])
print('-----'*10)
print('유출인구 순 내림차순')
print(df_ddong.sort_values('유출', ascending= False)['지역'][:10])
print('4위부터 순서 다름')

유입인구 순 내림차순
0    수원시 권선구 권선1동
1    수원시 영통구 매탄3동
2     수원시 영통구 원천동
3    수원시 팔달구 우만2동
4    수원시 영통구 매탄1동
5    수원시 영통구 광교1동
6     수원시 팔달구 행궁동
7    수원시 권선구 권선2동
8     수원시 권선구 곡선동
9      수원시 팔달구 지동
Name: 지역, dtype: object
--------------------------------------------------
유출인구 순 내림차순
0    수원시 권선구 권선1동
1    수원시 영통구 매탄3동
2     수원시 영통구 원천동
3    수원시 팔달구 우만2동
4     수원시 팔달구 행궁동
5    수원시 영통구 매탄1동
6    수원시 영통구 광교1동
7    수원시 권선구 권선2동
8     수원시 권선구 곡선동
9      수원시 팔달구 지동
Name: 지역, dtype: object
4위부터 순서 다름


# 3. 이동목적별 분석

## 3.1. 이동목적별 건수

In [99]:
purpose_cols = ['기타', '귀가', '출근', '등교', '귀사', '쇼핑', '관광', '병원']

In [100]:
df_cnt = datadf.pivot_table(index = 'purpose', columns = '이동구분', values = 'cnt', aggfunc = 'sum')
df_cnt = round(df_cnt, 2)
df_cnt = df_cnt.reset_index()

# 데이터프레임 정렬
df_cnt['purpose'] = pd.Categorical(df_cnt['purpose'], categories=purpose_cols, ordered=True)
df_cnt = df_cnt.sort_values('purpose')
df_cnt = df_cnt.reset_index(drop = True)

In [101]:
make_groupbar(df_cnt, 'purpose', ['유입', '유출', '동일지역간이동'], '이동목적별 유입, 유출 건수 합계', '유입, 유출 합계')

**유입**: 기타, 귀가, 출근, 등교, 귀사 순  
**유출**: 기타, 귀가, 출근, 등교, 귀사, 쇼핑, 관광, 병원 순  
**동일지역간이동**: 기타, 귀가, 출근, 귀사, 등교 순

**유출** 유출인구만 쇼핑, 관광, 병원 목적의 유출만 있음  
**유입** 쇼핑, 관광, 병원 목적의 유입인구는 없음

## 3.2. 일별 이동목적별 분석

In [102]:
purpose_cols2 = ['기타', '귀가', '출근', '등교', '귀사']

In [103]:
def make_df(move, idxcol):
    df_idx = datadf[datadf['이동구분']==move].pivot_table(index = idxcol, columns = 'purpose', values = 'cnt', aggfunc = 'sum')
    df_idx = round(df_idx, 2).reset_index()
    return df_idx

In [104]:
df_in = make_df('유입', 'etl_ymd')
df_out = make_df('유출', 'etl_ymd')

In [105]:
df_in = df_in[['etl_ymd']+purpose_cols2]
df_out =  df_out[['etl_ymd']+purpose_cols]

In [106]:
c = (
    Line()
    .add_xaxis(df_in['etl_ymd'].astype(str).tolist())
    .add_yaxis(
        "기타", df_in['기타'].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )
    .add_yaxis(
        "귀가",
        df_in['귀가'].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )
    .add_yaxis(
        "출근",
        df_in['출근'].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )
    .add_yaxis(
        "귀사",
        df_in['귀사'].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )
    .add_yaxis(
        "등교",
        df_in['등교'].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="이동목적별 유입건수 합계", subtitle='유입만'),
                     datazoom_opts = opts.DataZoomOpts())
    # .render("line_markpoint.html")
)
c.render_notebook()

**유입**
기타목적과 귀가, 출근, 귀사, 등교 목적의 일별 이동 패턴이 반대의 모습을 보임.

In [107]:
tempdf = df_out

xaxis_data = tempdf['etl_ymd'].astype(str).tolist()

# Line 차트 객체 생성
c = Line()

# x축 데이터 추가
c.add_xaxis(xaxis_data)

# 여러 y축 데이터 추가
yaxis_columns = purpose_cols  # 차트에 추가하고 싶은 컬럼 이름 목록
for col in yaxis_columns:
    c.add_yaxis(
        series_name=col, 
        y_axis=tempdf[col].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )

# 글로벌 옵션 설정
c.set_global_opts(
    title_opts=opts.TitleOpts(title="이동목적별 일별 유입건수 합계", subtitle='유출만'),
    legend_opts=opts.LegendOpts(orient="horizontal", pos_top="top"),
    datazoom_opts=opts.DataZoomOpts()
)

# 주피터 노트북에서 차트 렌더링
c.render_notebook()

**유출** 기타목적과 귀가목적의 일별 이동 패턴이 비슷한 모습을 보임.

### 3.2.1. 기타목적과 귀가목적만 유입유출 같이 보기

In [108]:
df_date_move_purpose = datadf.pivot_table(index = 'etl_ymd', columns = ['이동구분', 'purpose'], values = 'cnt', aggfunc = 'sum')
df_date_move_purpose = round(df_date_move_purpose, 2)
df_date_move_purpose.head()

이동구분      동일지역간이동                                       유입                   \
purpose        귀가      귀사       기타     등교      출근       귀가      귀사       기타   
etl_ymd                                                                       
20230601 11924.68 1883.02 19733.31 608.28 5311.37 17644.51 1380.29 46074.78   
20230602 12268.15 1766.56 21975.28 735.38 5347.17 17298.76 1160.87 49935.53   
20230603 12116.27  551.33 20813.59 276.03 2988.96 14053.95  359.40 62153.32   
20230604 11317.61  415.39 18207.25 232.30 2403.39 13967.86  236.30 44949.65   
20230605 12092.29 1555.91 20372.36 451.36 4874.14 16075.15 1040.25 51114.17   

이동구분                          유출                                           \
purpose       등교       출근     관광       귀가      귀사       기타      등교     병원   
etl_ymd                                                                     
20230601 1431.46 10747.96 231.10 27246.75 1394.29 34021.28 2383.29 108.57   
20230602 1599.96 11143.01 271.29 26692.97 1277.21 36353.64 2288.82  80.04   
20230603  261.05  4665.64 286.79 28705.85  406.02 42739.57  597.22  57.74   
20230604  166.10  2715.64 314.20 24292.63  345.12 34358.01  529.62  25.73   
20230605 1166.86  9584.87 288.33 27211.45 1139.64 36163.11 1460.75 116.70   

이동구분                     
purpose      쇼핑      출근  
etl_ymd                  
20230601 285.40 9031.08  
20230602 333.92 8950.06  
20230603 744.30 3395.72  
20230604 687.43 2380.69  
20230605 492.58 7425.08

In [109]:
_list = []
for col in df_date_move_purpose.columns:
    _list.append(col[0] + '_'+ col[1])

df_date_move_purpose.columns = _list

In [110]:
df_date_move_purpose = df_date_move_purpose.reset_index()

In [112]:
df_date_move_purpose.columns

Index(['etl_ymd', '동일지역간이동_귀가', '동일지역간이동_귀사', '동일지역간이동_기타', '동일지역간이동_등교',
       '동일지역간이동_출근', '유입_귀가', '유입_귀사', '유입_기타', '유입_등교', '유입_출근', '유출_관광',
       '유출_귀가', '유출_귀사', '유출_기타', '유출_등교', '유출_병원', '유출_쇼핑', '유출_출근'],
      dtype='object')

In [113]:
tempdf = df_date_move_purpose

xaxis_data = tempdf['etl_ymd'].astype(str).tolist()

# Line 차트 객체 생성
c = Line()

# x축 데이터 추가
c.add_xaxis(xaxis_data)

# 여러 y축 데이터 추가
yaxis_columns = ['유입_기타', '유입_귀가', '유출_기타', '유출_귀가', '동일지역간이동_기타', '동일지역간이동_귀가']  # 차트에 추가하고 싶은 컬럼 이름 목록
for col in yaxis_columns:
    c.add_yaxis(
        series_name=col, 
        y_axis=tempdf[col].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )

# 글로벌 옵션 설정
c.set_global_opts(
    title_opts=opts.TitleOpts(title="이동목적별 유입유출건수 합계", subtitle='유입, 유출 합계'),
    legend_opts=opts.LegendOpts(orient="horizontal", pos_top="top"),
    datazoom_opts=opts.DataZoomOpts()
)

# 주피터 노트북에서 차트 렌더링
c.render_notebook()

**기타** 기타 목적의 경우 유입과 유출의 패턴이 비슷함.  
**귀가** 귀가 목적의 경우 유입과 유출의 패턴이 반대.

### 3.2.2 캘린더차트로 일별보기 - 유입

In [114]:
purpose_cols

['기타', '귀가', '출근', '등교', '귀사', '쇼핑', '관광', '병원']

In [115]:
from pyecharts.charts import Calendar, Grid
from pyecharts import options as opts
import datetime

# 시작일과 종료일 정의
begin = datetime.date(2023, 6, 1)
end = datetime.date(2023, 6, 30)

# 캘린더 차트 생성 함수
def create_calendar_chart(data, title, df, purpose):
    return (
        Calendar()
        .add(
            series_name="",
            yaxis_data=data,
            calendar_opts=opts.CalendarOpts(
                range_="2023-06",
                pos_top="120",
                pos_left="30",
                pos_right="30",
                yearlabel_opts=opts.CalendarYearLabelOpts(is_show=False),
            ),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title=title, pos_top="10%"),
            visualmap_opts=opts.VisualMapOpts(
                max_=df[purpose].max(), 
                min_=df[purpose].min(),
                orient="horizontal", 
                is_piecewise=False,
                pos_top="0%"
            ),
        )
    )


In [116]:
purpose = '기타'
data = [
    [str(begin + datetime.timedelta(days=i)), df_in['기타'][i]]
    for i in range((end - begin).days + 1)
]
create_calendar_chart(data, f"이동목적별 유입건수 합계 - {purpose}", df_in, '기타').render_notebook()

In [117]:
purpose = '귀가'
data = [
    [str(begin + datetime.timedelta(days=i)), df_in[purpose][i]]
    for i in range((end - begin).days + 1)
]
create_calendar_chart(data, f"이동목적별 유입건수 합계 - {purpose}", df_in, purpose).render_notebook()

In [118]:
purpose = '출근'
data = [
    [str(begin + datetime.timedelta(days=i)), df_in[purpose][i]]
    for i in range((end - begin).days + 1)
]
create_calendar_chart(data, f"이동목적별 유입건수 합계 - {purpose}", df_in, purpose).render_notebook()

In [119]:
purpose = '등교'
data = [
    [str(begin + datetime.timedelta(days=i)), df_in[purpose][i]]
    for i in range((end - begin).days + 1)
]
create_calendar_chart(data, f"이동목적별 유입건수 합계 - {purpose}", df_in, purpose).render_notebook()

In [120]:
purpose = '귀사'
data = [
    [str(begin + datetime.timedelta(days=i)), df_in[purpose][i]]
    for i in range((end - begin).days + 1)
]
create_calendar_chart(data, f"이동목적별 유입건수 합계 - {purpose}", df_in, purpose).render_notebook()

### 3.2.2 캘린더차트로 일별보기 - 유출

In [121]:
purpose = '기타'
data = [
    [str(begin + datetime.timedelta(days=i)), df_out['기타'][i]]
    for i in range((end - begin).days + 1)
]
create_calendar_chart(data, f"이동목적별 유입건수 합계 - {purpose}", df_out, '기타').render_notebook()

In [122]:
purpose = '귀가'
data = [
    [str(begin + datetime.timedelta(days=i)), df_out[purpose][i]]
    for i in range((end - begin).days + 1)
]
create_calendar_chart(data, f"이동목적별 유입건수 합계 - {purpose}", df_out, purpose).render_notebook()

In [123]:
purpose = '출근'
data = [
    [str(begin + datetime.timedelta(days=i)), df_out[purpose][i]]
    for i in range((end - begin).days + 1)
]
create_calendar_chart(data, f"이동목적별 유입건수 합계 - {purpose}", df_out, purpose).render_notebook()

In [124]:
purpose = '등교'
data = [
    [str(begin + datetime.timedelta(days=i)), df_out[purpose][i]]
    for i in range((end - begin).days + 1)
]
create_calendar_chart(data, f"이동목적별 유입건수 합계 - {purpose}", df_out, purpose).render_notebook()

In [125]:
purpose = '귀사'
data = [
    [str(begin + datetime.timedelta(days=i)), df_out[purpose][i]]
    for i in range((end - begin).days + 1)
]
create_calendar_chart(data, f"이동목적별 유입건수 합계 - {purpose}", df_out, purpose).render_notebook()

## 3.3. 요일별 이동목적별 분석

In [126]:
df_dow_in = make_df('유입', 'dow')
df_dow_out = make_df('유출', 'dow')

In [127]:
# 요일 순서 정의 (월요일부터 일요일까지)
dow_order = ['월', '화', '수', '목', '금', '토', '일']

# 요일 순서에 따라 데이터프레임 정렬
df_dow_in['dow'] = pd.Categorical(df_dow_in['dow'], categories=dow_order, ordered=True)
df_sorted_in = df_dow_in.sort_values('dow')
df_sorted_in = df_sorted_in.reset_index(drop = True)

# 요일 순서에 따라 데이터프레임 정렬
df_dow_out['dow'] = pd.Categorical(df_dow_out['dow'], categories=dow_order, ordered=True)
df_sorted_out = df_dow_out.sort_values('dow')
df_sorted_out = df_sorted_out.reset_index(drop = True)

In [128]:
tempdf = df_sorted_in

xaxis_data = tempdf['dow'].astype(str).tolist()

# Line 차트 객체 생성
c = Line()

# x축 데이터 추가
c.add_xaxis(xaxis_data)

# 여러 y축 데이터 추가
yaxis_columns = purpose_cols2  # 차트에 추가하고 싶은 컬럼 이름 목록
for col in yaxis_columns:
    c.add_yaxis(
        series_name=col, 
        y_axis=tempdf[col].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )

# 글로벌 옵션 설정
c.set_global_opts(
    title_opts=opts.TitleOpts(title="이동목적별 요일별 유입건수 합계", subtitle='유입만'),
    legend_opts=opts.LegendOpts(orient="horizontal", pos_top="top"),
    # datazoom_opts=opts.DataZoomOpts()
)

# 주피터 노트북에서 차트 렌더링
c.render_notebook()

In [129]:
tempdf = df_sorted_out

xaxis_data = tempdf['dow'].astype(str).tolist()

# Line 차트 객체 생성
c = Line()

# x축 데이터 추가
c.add_xaxis(xaxis_data)

# 여러 y축 데이터 추가
yaxis_columns = purpose_cols  # 차트에 추가하고 싶은 컬럼 이름 목록
for col in yaxis_columns:
    c.add_yaxis(
        series_name=col, 
        y_axis=tempdf[col].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )

# 글로벌 옵션 설정
c.set_global_opts(
    title_opts=opts.TitleOpts(title="이동목적별 요일별 유출건수 합계", subtitle='유출만'),
    legend_opts=opts.LegendOpts(orient="horizontal", pos_top="top"),
    # datazoom_opts=opts.DataZoomOpts()
)

# 주피터 노트북에서 차트 렌더링
c.render_notebook()

### 3.3.1 기타, 귀가목적만 유입, 유출 함께 보기

In [130]:
df_dow_move_purpose = datadf.pivot_table(index = 'dow', columns = ['이동구분', 'purpose'], values = 'cnt', aggfunc = 'sum')
df_dow_move_purpose = round(df_dow_move_purpose, 2)

In [131]:
_list = []
for col in df_dow_move_purpose.columns:
    _list.append(col[0] + '_'+ col[1])

df_dow_move_purpose.columns = _list
df_dow_move_purpose = df_dow_move_purpose.reset_index()

# 요일 순서에 따라 데이터프레임 정렬
df_dow_move_purpose['dow'] = pd.Categorical(df_dow_move_purpose['dow'], categories=dow_order, ordered=True)
df_dow_move_purpose = df_dow_move_purpose.sort_values('dow')
df_dow_move_purpose = df_dow_move_purpose.reset_index(drop = True)

In [132]:
df_dow_move_purpose.columns

Index(['dow', '동일지역간이동_귀가', '동일지역간이동_귀사', '동일지역간이동_기타', '동일지역간이동_등교',
       '동일지역간이동_출근', '유입_귀가', '유입_귀사', '유입_기타', '유입_등교', '유입_출근', '유출_관광',
       '유출_귀가', '유출_귀사', '유출_기타', '유출_등교', '유출_병원', '유출_쇼핑', '유출_출근'],
      dtype='object')

In [133]:
tempdf = df_dow_move_purpose

xaxis_data = tempdf['dow'].astype(str).tolist()

# Line 차트 객체 생성
c = Line()

# x축 데이터 추가
c.add_xaxis(xaxis_data)

# 여러 y축 데이터 추가
yaxis_columns = ['유입_기타', '유입_귀가', '유출_기타', '유출_귀가']  # 차트에 추가하고 싶은 컬럼 이름 목록
for col in yaxis_columns:
    c.add_yaxis(
        series_name=col, 
        y_axis=tempdf[col].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )

# 글로벌 옵션 설정
c.set_global_opts(
    title_opts=opts.TitleOpts(title="이동목적별 요일별 유입유출건수 합계", subtitle='유입, 유출 건수'),
    legend_opts=opts.LegendOpts(orient="horizontal", pos_top="top"),
    # datazoom_opts=opts.DataZoomOpts()
)

# 주피터 노트북에서 차트 렌더링
c.render_notebook()

## 3.3. 시간대별 이동목적별 분석

### 3.3.1. 출발시간대별 이동목적별 분석

In [134]:
df_otime_in = make_df('유입', 'o_time_cd')
df_otime_out = make_df('유출', 'o_time_cd')

In [135]:
df_otime_in = df_otime_in[['o_time_cd']+purpose_cols2]
df_otime_out = df_otime_out[['o_time_cd']+purpose_cols]

In [136]:
tempdf = df_otime_in
title_txt = '출발시간대별 이동목적별 유입건수'
subtitle_txt = '유입만'
xcol = 'o_time_cd'
ycols = purpose_cols2

xaxis_data = tempdf[xcol].astype(str).tolist()

# Line 차트 객체 생성
c = Line()

# x축 데이터 추가
c.add_xaxis(xaxis_data)

# 여러 y축 데이터 추가
yaxis_columns = ycols  # 차트에 추가하고 싶은 컬럼 이름 목록
for col in yaxis_columns:
    c.add_yaxis(
        series_name=col, 
        y_axis=tempdf[col].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )

# 글로벌 옵션 설정
c.set_global_opts(
    title_opts=opts.TitleOpts(title=title_txt, subtitle=subtitle_txt),
    legend_opts=opts.LegendOpts(orient="horizontal", pos_top="top"),
    # datazoom_opts=opts.DataZoomOpts()
)

# 주피터 노트북에서 차트 렌더링
c.render_notebook()

In [137]:
tempdf = df_otime_out
title_txt = '출발시간대별 이동목적별 유출건수'
subtitle_txt = '유출만'
xcol = 'o_time_cd'
ycols = purpose_cols

xaxis_data = tempdf[xcol].astype(str).tolist()

# Line 차트 객체 생성
c = Line()

# x축 데이터 추가
c.add_xaxis(xaxis_data)

# 여러 y축 데이터 추가
yaxis_columns = ycols  # 차트에 추가하고 싶은 컬럼 이름 목록
for col in yaxis_columns:
    c.add_yaxis(
        series_name=col, 
        y_axis=tempdf[col].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )

# 글로벌 옵션 설정
c.set_global_opts(
    title_opts=opts.TitleOpts(title=title_txt, subtitle=subtitle_txt),
    legend_opts=opts.LegendOpts(orient="horizontal", pos_top="top"),
    # datazoom_opts=opts.DataZoomOpts()
)

# 주피터 노트북에서 차트 렌더링
c.render_notebook()

### 3.3.2. 도착시간대별 이동목적별 분석

In [138]:
df_dtime_in = make_df('유입', 'd_time_cd')
df_dtime_out = make_df('유출', 'd_time_cd')

In [139]:
df_dtime_in = df_dtime_in[['d_time_cd']+purpose_cols2]
df_dtime_out = df_dtime_out[['d_time_cd']+purpose_cols]

In [140]:
tempdf = df_dtime_in
title_txt = '도착시간대별 이동목적별 유입건수'
subtitle_txt = '유입만'
xcol = 'd_time_cd'
ycols = purpose_cols2

xaxis_data = tempdf[xcol].astype(str).tolist()

# Line 차트 객체 생성
c = Line()

# x축 데이터 추가
c.add_xaxis(xaxis_data)

# 여러 y축 데이터 추가
yaxis_columns = ycols  # 차트에 추가하고 싶은 컬럼 이름 목록
for col in yaxis_columns:
    c.add_yaxis(
        series_name=col, 
        y_axis=tempdf[col].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )

# 글로벌 옵션 설정
c.set_global_opts(
    title_opts=opts.TitleOpts(title=title_txt, subtitle=subtitle_txt),
    legend_opts=opts.LegendOpts(orient="horizontal", pos_top="top"),
    # datazoom_opts=opts.DataZoomOpts()
)

# 주피터 노트북에서 차트 렌더링
c.render_notebook()

In [141]:
tempdf = df_dtime_out
title_txt = '도착시간대별 이동목적별 유출건수'
subtitle_txt = '유출만'
xcol = 'd_time_cd'
ycols = purpose_cols

xaxis_data = tempdf[xcol].astype(str).tolist()

# Line 차트 객체 생성
c = Line()

# x축 데이터 추가
c.add_xaxis(xaxis_data)

# 여러 y축 데이터 추가
yaxis_columns = ycols  # 차트에 추가하고 싶은 컬럼 이름 목록
for col in yaxis_columns:
    c.add_yaxis(
        series_name=col, 
        y_axis=tempdf[col].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )

# 글로벌 옵션 설정
c.set_global_opts(
    title_opts=opts.TitleOpts(title=title_txt, subtitle=subtitle_txt),
    legend_opts=opts.LegendOpts(orient="horizontal", pos_top="top"),
    # datazoom_opts=opts.DataZoomOpts()
)

# 주피터 노트북에서 차트 렌더링
c.render_notebook()

## 3.4. 성별 이동인구 분석

In [142]:
df_sex_in = make_df('유입', 'sex_type_itg_cd')
df_sex_out = make_df('유출', 'sex_type_itg_cd')

df_sex_in = df_sex_in[['sex_type_itg_cd']+purpose_cols2]
df_sex_out = df_sex_out[['sex_type_itg_cd']+purpose_cols]

In [143]:
df_sex_in = df_sex_in.set_index('sex_type_itg_cd').T.reset_index()
df_sex_out = df_sex_out.set_index('sex_type_itg_cd').T.reset_index()

In [144]:
make_groupbar(df_sex_in, 'purpose', ['M','F'], '성별 이동목적별 유입인구', '유입만')

In [145]:
make_groupbar(df_sex_out, 'purpose', ['M','F'], '성별 이동목적별 유출인구', '유출만')

## 3.5. 연령별 이동목적별 인구

In [146]:
df_age_in = make_df('유입', 'age')
df_age_out = make_df('유출', 'age')

df_age_in = df_age_in[['age']+purpose_cols2]
df_age_out = df_age_out[['age']+purpose_cols]

In [147]:
tempdf = df_age_in
title_txt = '연령별 이동목적별 유입건수'
subtitle_txt = '유입만'
xcol = 'age'
ycols = purpose_cols2

xaxis_data = tempdf[xcol].astype(str).tolist()

# Line 차트 객체 생성
c = Line()

# x축 데이터 추가
c.add_xaxis(xaxis_data)

# 여러 y축 데이터 추가
yaxis_columns = ycols  # 차트에 추가하고 싶은 컬럼 이름 목록
for col in yaxis_columns:
    c.add_yaxis(
        series_name=col, 
        y_axis=tempdf[col].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )

# 글로벌 옵션 설정
c.set_global_opts(
    title_opts=opts.TitleOpts(title=title_txt, subtitle=subtitle_txt),
    legend_opts=opts.LegendOpts(orient="horizontal", pos_top="top"),
    # datazoom_opts=opts.DataZoomOpts()
)

# 주피터 노트북에서 차트 렌더링
c.render_notebook()

In [148]:
tempdf = df_age_out
title_txt = '연령별 이동목적별 유출건수'
subtitle_txt = '유출만'
xcol = 'age'
ycols = purpose_cols

xaxis_data = tempdf[xcol].astype(str).tolist()

# Line 차트 객체 생성
c = Line()

# x축 데이터 추가
c.add_xaxis(xaxis_data)

# 여러 y축 데이터 추가
yaxis_columns = ycols  # 차트에 추가하고 싶은 컬럼 이름 목록
for col in yaxis_columns:
    c.add_yaxis(
        series_name=col, 
        y_axis=tempdf[col].tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),
    )

# 글로벌 옵션 설정
c.set_global_opts(
    title_opts=opts.TitleOpts(title=title_txt, subtitle=subtitle_txt),
    legend_opts=opts.LegendOpts(orient="horizontal", pos_top="top"),
    # datazoom_opts=opts.DataZoomOpts()
)

# 주피터 노트북에서 차트 렌더링
c.render_notebook()

## 3.6. 유입/유출지별 이동목적별 이동

### 3.6.1. 시도

In [149]:
df_sd_in = make_df('유입', 'o_sido_nm')
df_sd_out = make_df('유출', 'd_sido_nm')

In [150]:
df_sd_in['total'] = df_sd_in[purpose_cols2].sum(axis = 1)
df_sd_out['total'] = df_sd_out[purpose_cols].sum(axis = 1)

df_sd_in = df_sd_in.sort_values('total', ascending = False).reset_index(drop = True)
df_sd_out = df_sd_out.sort_values('total', ascending = False).reset_index(drop = True)

df_sd_in = df_sd_in[['o_sido_nm']+purpose_cols2]
df_sd_out = df_sd_out[['d_sido_nm']+purpose_cols]

In [151]:
make_groupbar(df_sd_in, 'o_sido_nm', purpose_cols2, '출발지역별 이동목적별 유입인구(시도)', '유입만')

In [152]:
make_groupbar(df_sd_out, 'd_sido_nm', purpose_cols, '도착지역별 이동목적별 유출인구(시도)', '유출만')

### 3.6.2. 시군구

In [153]:
df_sgg_in = make_df('유입', 'o_sgg_nm')
df_sgg_out = make_df('유출', 'd_sgg_nm')

In [154]:
df_sgg_in['total'] = df_sgg_in[purpose_cols2].sum(axis = 1)
df_sgg_out['total'] = df_sgg_out[purpose_cols].sum(axis = 1)

df_sgg_in = df_sgg_in.sort_values('total', ascending = False).reset_index(drop = True)
df_sgg_out = df_sgg_out.sort_values('total', ascending = False).reset_index(drop = True)

df_sgg_in = df_sgg_in[['o_sgg_nm']+purpose_cols2]
df_sgg_out = df_sgg_out[['d_sgg_nm']+purpose_cols]

In [155]:
make_groupbar(df_sgg_in, 'o_sgg_nm', purpose_cols2, '출발지역별 이동목적별 유입인구(시군구)', '유입만')

In [156]:
make_groupbar(df_sgg_out, 'd_sgg_nm', purpose_cols, '도착지역별 이동목적별 유출인구(시군구)', '유출만')

### 3.6.3. 읍면동

In [157]:
datadf['o_dong'] = datadf['o_sgg_nm'] +' '+datadf['o_admdong_nm']
datadf['d_dong'] = datadf['d_sgg_nm'] +' '+datadf['d_admdong_nm']

In [158]:
df_dong_in = make_df('유입', 'o_dong')
df_dong_out = make_df('유출', 'd_dong')

In [159]:
df_dong_in['total'] = df_dong_in[purpose_cols2].sum(axis = 1)
df_dong_out['total'] = df_dong_out[purpose_cols].sum(axis = 1)

df_dong_in = df_dong_in.sort_values('total', ascending = False).reset_index(drop = True)
df_dong_out = df_dong_out.sort_values('total', ascending = False).reset_index(drop = True)

df_dong_in = df_dong_in[['o_dong']+purpose_cols2]
df_dong_out = df_dong_out[['d_dong']+purpose_cols]

In [160]:
df_dong_in = df_dong_in[:50]
df_dong_out = df_dong_out[:50]

In [161]:
make_groupbar(df_dong_in, 'o_dong', purpose_cols2, '출발지역별 이동목적별 유입인구(행정동)', '유입만')

In [162]:
make_groupbar(df_dong_out, 'd_dong', purpose_cols2, '도착지역별 이동목적별 유입인구(행정동)', '유출만')

## 4. 이동목적 개별 분석

In [163]:
purpose_cols

['기타', '귀가', '출근', '등교', '귀사', '쇼핑', '관광', '병원']

In [164]:
datadf.head()

,etl_ymd,sex_type_itg_cd,age_gr,in_forn_div_nm,o_time_cd,d_time_cd,o_admi_cd,d_admi_cd,purpose,avg_dist_meter,...,age,sexage,o_sido_nm,o_sgg_nm,o_admdong_nm,d_sido_nm,d_sgg_nm,d_admdong_nm,o_dong,d_dong
0,20230624,F,35,내국인,15,16,41115730.00,41410580.00,기타,12166.00,...,30,F30,경기도,수원시 팔달구,인계동,경기도,군포시,오금동,수원시 팔달구 인계동,군포시 오금동
1,20230630,F,35,내국인,11,11,41115710.00,41115730.00,기타,1570.00,...,30,F30,경기도,수원시 팔달구,우만1동,경기도,수원시 팔달구,인계동,수원시 팔달구 우만1동,수원시 팔달구 인계동
2,20230627,M,20,내국인,23,23,41113670.00,41115730.00,귀가,461.00,...,20,M20,경기도,수원시 권선구,권선1동,경기도,수원시 팔달구,인계동,수원시 권선구 권선1동,수원시 팔달구 인계동
3,20230616,F,55,내국인,8,9,41463590.00,41115730.00,출근,7255.00,...,50,F50,경기도,용인시 기흥구,보정동,경기도,수원시 팔달구,인계동,용인시 기흥구 보정동,수원시 팔달구 인계동
4,20230617,M,35,내국인,18,19,41117520.00,41115730.00,기타,1807.00,...,30,M30,경기도,수원시 영통구,매탄2동,경기도,수원시 팔달구,인계동,수원시 영통구 매탄2동,수원시 팔달구 인계동


In [167]:
!jupyter nbconvert flow_with_purpose_240313.ipynb --to slides --post serve

[NbConvertApp] Converting notebook flow_with_purpose_240313.ipynb to slides
[NbConvertApp] Writing 1537753 bytes to flow_with_purpose_240313.slides.html
[NbConvertApp] Redirecting reveal.js requests to https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.5.0
Traceback (most recent call last):
  File "C:\Users\sjpark\anaconda3\envs\nemo\Scripts\jupyter-nbconvert-script.py", line 10, in <module>
    sys.exit(main())
  File "C:\Users\sjpark\anaconda3\envs\nemo\lib\site-packages\jupyter_core\application.py", line 264, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "C:\Users\sjpark\anaconda3\envs\nemo\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\sjpark\anaconda3\envs\nemo\lib\site-packages\nbconvert\nbconvertapp.py", line 346, in start
    self.convert_notebooks()
  File "C:\Users\sjpark\anaconda3\envs\nemo\lib\site-packages\nbconvert\nbconvertapp.py", line 518, in convert_no